# About this Dataset

Welcome to New York City, one of the most-visited cities in the world. There are many Airbnb listings in New York City to meet the high demand for temporary lodging for travelers, which can be anywhere between a few nights to many months. In this project, we will take a closer look at the New York Airbnb market by combining data from multiple file types like .csv, .tsv, and .xlsx.

Recall that **CSV, TSV**, and **Excel** files are three common formats for storing data. Three files containing data on 2019 Airbnb listings are available to you:

**data/airbnb_price.csv** This is a CSV file containing data on Airbnb listing prices and locations.

- listing_id: unique identifier of listing
- price: nightly listing price in USD
- nbhood_full: name of borough and neighborhood where listing is located

**data/airbnb_room_type.xlsx** This is an Excel file containing data on Airbnb listing descriptions and room types.

- listing_id: unique identifier of listing
- description: listing description
- room_type: Airbnb has three types of rooms: shared rooms, private rooms, and entire homes/apartments

**data/airbnb_last_review.tsv** This is a TSV file containing data on Airbnb host names and review dates.

- listing_id: unique identifier of listing
- host_name: name of listing host
- last_review: date when the listing was last reviewed

# Table of contents

As a consultant working for a real estate start-up, you have collected Airbnb listing data from various sources to investigate the short-term rental market in New York. You'll analyze this data to provide insights on private rooms to the real estate company.

There are three files in the data folder: <code>airbnb_price.csv</code>, <code>airbnb_room_type.xlsx</code>, <code>airbnb_last_review.tsv</code>.

- What are the dates of the earliest and most recent reviews? Store these values as two separate variables with your preferred names.
- How many of the listings are private rooms? Save this into any variable.
- What is the average listing price? Round to the nearest penny and save into a variable.
- Combine the new variables into one DataFrame called <code>review_dates</code> with four columns in the following order: <code>first_reviewed</code>, <code>last_reviewed</code>, <code>nb_private_rooms</code>, and <code>avg_price</code>. The DataFrame should only contain one row of values.

In [36]:
!pip install opendatasets

In [37]:
import opendatasets as od

In [38]:
# Mount google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [39]:
#Copy credencials of kaggle to correct place
!mkdir -p ~/.kaggle
!cp '/content/drive/MyDrive/Colab Notebooks/kaggle.json' ~/.kaggle/
!cp '/content/drive/MyDrive/Colab Notebooks/kaggle.json' ./
!chmod 600 ~/.kaggle/kaggle.json
print("ok")

ok


In [40]:
# create folder of kaggle with url of dataset
od.download(
    "https://www.kaggle.com/datasets/frandlt/nyc-airbnb-market")

Skipping, found downloaded files in "./nyc-airbnb-market" (use force=True to force download)


In [41]:
# create folder of kaggle with url of dataset
od.download(
    "https://www.kaggle.com/datasets/aynashairi/rooms-type")

Skipping, found downloaded files in "./rooms-type" (use force=True to force download)


# Import Libraries

In [42]:
import pandas as pd
import numpy as np


import warnings
warnings.filterwarnings("ignore")

# Load data

In [43]:
# Read data (cvs file)

file =('/content/nyc-airbnb-market/airbnb_last_review.tsv')
reviews = pd.read_csv(file, sep='\t')
# , index_col=0


file1 =('/content/nyc-airbnb-market/airbnb_price.csv')
prices = pd.read_csv(file1)

reviews.head(), prices.head()


# room_type.parse(0)

(   listing_id    host_name   last_review
 0        2595     Jennifer   May 21 2019
 1        3831  LisaRoxanne  July 05 2019
 2        5099        Chris  June 22 2019
 3        5178     Shunichi  June 24 2019
 4        5238          Ben  June 09 2019,
    listing_id        price                nbhood_full
 0        2595  225 dollars         Manhattan, Midtown
 1        3831   89 dollars     Brooklyn, Clinton Hill
 2        5099  200 dollars     Manhattan, Murray Hill
 3        5178   79 dollars  Manhattan, Hell's Kitchen
 4        5238  150 dollars       Manhattan, Chinatown)

In [44]:
file2 =('/content/rooms-type/airbnb_room_type.xlsx')
room_types = pd.read_excel(file2, sheet_name=0)

room_types.head()

,listing_id,description,room_type
0,2595,Skylit Midtown Castle,Entire home/apt
1,3831,Cozy Entire Floor of Brownstone,Entire home/apt
2,5099,Large Cozy 1 BR Apartment In Midtown East,Entire home/apt
3,5178,Large Furnished Room Near B'way,private room
4,5238,Cute & Cozy Lower East Side 1 bdrm,Entire home/apt


## clean price column

In [45]:
# Remove whitespace and string characters from prices column
prices['price']= prices['price'].str.replace(' dollars', '')

# convert prices column to numeric datatype
prices['price']=pd.to_numeric(prices['price'])

#print 3 rows
prices.head(3)

,listing_id,price,nbhood_full
0,2595,225,"Manhattan, Midtown"
1,3831,89,"Brooklyn, Clinton Hill"
2,5099,200,"Manhattan, Murray Hill"


## descriptive stat

In [46]:
print(prices.describe())

         listing_id         price
count  2.520900e+04  25209.000000
mean   2.068922e+07    141.777936
std    1.102928e+07    147.349137
min    2.595000e+03      0.000000
25%    1.202273e+07     69.000000
50%    2.234391e+07    105.000000
75%    3.037669e+07    175.000000
max    3.645581e+07   7500.000000


- We can see three quarters of listings cost $175 per night or less.

- However, there are some outliers including a maximum price of $7,500 per night.

Some of listings are actually showing as free. Let's remove these from the DataFrame, and calculate the average price.

## Calc the average

In [47]:
# subset the listing where price is $0, free_listings

free_listing = (prices['price']== 0)

# update file prices removing the prices where free listing
prices = prices.loc[~free_listing]

# calculate the average price, avg_price
avg_price= round(prices['price'].mean(), 2)

#print the avg
print(f'The average price per night for an Airbnb Listing in NYC is ${avg_price}')


The average price per night for an Airbnb Listing in NYC is $141.82


## Comparing costs to the private rental market

So the avregare listing cost is on average, per night, €142.
So let's do a benchmark for comparison.

According to Zumper, a 1 bedroom apartment in New York City costs, on average, $3,100 per month. Let's convert the per night prices of our listings into monthly costs, so we can compare to the private market.

In [48]:
#compare cost to private rental market

# add a column to prices to have the price by month
prices['price_monthly']=prices['price']*365/12

# calculate avg_price monthly
avg_price_monthly=round(prices['price_monthly'].mean(), 2)

# Print the average price monthly
print(f'The average price monthly for an Airbnb Listing in NYC is ${avg_price_monthly} and in private market would be around $3100.')

The average price monthly for an Airbnb Listing in NYC is $4313.61 and in private market would be around $3100.


## clean the room_type column

In [49]:
# check unique values
room_types['room_type'].unique()

array(['Entire home/apt', 'private room', 'Private room',
       'entire home/apt', 'PRIVATE ROOM', 'shared room',
       'ENTIRE HOME/APT', 'Shared room', 'SHARED ROOM'], dtype=object)

In [50]:
# Convert the room_type column to lowercase
room_types["room_type"] = room_types["room_type"].str.lower()

# Update the room_type column to category data type
room_types["room_type"] = room_types["room_type"].astype("category")

# Create the variable room_frequencies
room_frequencies = room_types["room_type"].value_counts()

# Print room_frequencies
print(room_frequencies)

# Filter a specific value from the value counts
value_to_filter = "private room"  # For example, filtering value 3
filtered_count = room_frequencies.get(value_to_filter, 0)  # If value_to_filter doesn't exist, return 0

# Display the filtered count
print(f"The count of {value_to_filter} is: {filtered_count}")

room_type
entire home/apt    13266
private room       11356
shared room          587
Name: count, dtype: int64
The count of private room is: 11356


What timeframe are we working with?

In [51]:
reviews['last_review']=pd.to_datetime(reviews['last_review'], format='%B %d %Y')

#create first review date
first_reviewed=reviews['last_review'].dt.date.min()

#create last review date, most recent
last_reviewed=reviews['last_review'].dt.date.max()

#print first and last review date
print(f'The latest Airbnb review is {first_reviewed} and the earliest review is {last_reviewed}')

The latest Airbnb review is 2019-01-01 and the earliest review is 2019-07-09


 ## Joining the DataFrames.

Now we've extracted the information needed, we will merge the three DataFrames to make any future analysis easier to conduct. Once we have joined the data, we will remove any observations with missing values and check for duplicates.

In [53]:
# Merge prices and room_types to create rooms_and_prices
# rooms_and_prices = prices.merge(room_types, how= 'outer', on= 'listing_id') # out to consider all rows
rooms_and_prices = pd.merge(prices, room_types,  on= 'listing_id') # out to consider all rows

# Merge rooms_and_prices with the reviews DataFrame to create airbnb_merged
# airbnb_merged = rooms_and_prices.merge (reviews, how='outer', on='listing_id') # out to consider all rows
airbnb_merged = pd.merge(rooms_and_prices, reviews, on='listing_id') # out to consider all rows

airbnb_merged.columns, airbnb_merged.shape

(Index(['listing_id', 'price', 'nbhood_full', 'price_monthly', 'description',
        'room_type', 'host_name', 'last_review'],
       dtype='object'),
 (25202, 8))

In [54]:
# Drop missing values from airbnb_merged
airbnb_merged.dropna(inplace=True)

# Check if there are any duplicate values
print("There are {} duplicates in the DataFrame.".format(airbnb_merged.duplicated().sum()))

There are 0 duplicates in the DataFrame.


## Analyzing listing prices by NYC borough
Now we have combined all data into a single DataFrame, we will turn our attention to understanding the difference in listing prices between New York City boroughs. We can currently see boroughs listed as the first part of a string within the <code>nbhood_full</code> column, e.g.,

In [55]:
airbnb_merged.head()

,listing_id,price,nbhood_full,price_monthly,description,room_type,host_name,last_review
0,2595,225,"Manhattan, Midtown",6843.750000,Skylit Midtown Castle,entire home/apt,Jennifer,2019-05-21
1,3831,89,"Brooklyn, Clinton Hill",2707.083333,Cozy Entire Floor of Brownstone,entire home/apt,LisaRoxanne,2019-07-05
2,5099,200,"Manhattan, Murray Hill",6083.333333,Large Cozy 1 BR Apartment In Midtown East,entire home/apt,Chris,2019-06-22
3,5178,79,"Manhattan, Hell's Kitchen",2402.916667,Large Furnished Room Near B'way,private room,Shunichi,2019-06-24
4,5238,150,"Manhattan, Chinatown",4562.500000,Cute & Cozy Lower East Side 1 bdrm,entire home/apt,Ben,2019-06-09


In [56]:
# Extract information from the nbhood_full column and store as a new column, borough
airbnb_merged["borough"] = airbnb_merged["nbhood_full"].str.partition(",")[0]

# Group by borough and calculate summary statistics
boroughs = airbnb_merged.groupby("borough")["price"].agg(["sum", "mean", "median", "count"])

# Round boroughs to 2 decimal places, and sort by mean in descending order
boroughs = boroughs.round(2).sort_values("mean", ascending= False)

# Print boroughs
print(boroughs.T)

borough   Manhattan    Brooklyn     Queens  Staten Island     Bronx
sum      1898417.00  1275250.00  320715.00       22974.00  55156.00
mean         184.04      122.02      92.83          86.04     79.25
median       149.00       95.00      70.00          71.00     65.00
count      10315.00    10451.00    3455.00         267.00    696.00


## Price range by borough
The above output gives us a summary of prices for listings across the 5 boroughs. In this final task we would like to categorize listings based on whether they fall into specific price ranges, and view this by borough.

We can do this using percentiles and labels to create a new column, price_range, in the DataFrame. Once we have created the labels, we can then group the data and count frequencies for listings in each price range by borough.

We will assign the following categories and price ranges:
- budget-> 0-69
- avg -> 70-175
- expensive -> 176-350
- Extravagant -> 350




In [57]:
#create labels for price range
Label_names=['Budget', 'Average', 'Expensive', 'Extravagant']

# create the label ranges
ranges = [0,69,175,350,np.inf]

# insert new column, price_range
airbnb_merged['price_range']= pd.cut(airbnb_merged['price'], bins = ranges, labels=Label_names)

# calc borought and price range frequencies, prices by borought
prices_by_borough= airbnb_merged.groupby(['borough', 'price_range'])['price_range'].count()

print(prices_by_borough)

borough        price_range
Bronx          Budget          381
               Average         285
               Expensive        25
               Extravagant       5
Brooklyn       Budget         3194
               Average        5532
               Expensive      1466
               Extravagant     259
Manhattan      Budget         1148
               Average        5285
               Expensive      3072
               Extravagant     810
Queens         Budget         1631
               Average        1505
               Expensive       291
               Extravagant      28
Staten Island  Budget          124
               Average         123
               Expensive        20
               Extravagant       0
Name: price_range, dtype: int64
